### Markov Chain

In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import numpy as np
import random


In [2]:
class MarkovChain:
    def __init__(self, t_matrix, stages, states, random_vars, probs):
        self.t_matrix = t_matrix
        self.stages = list(range(1,stages+1))
        self.states = list(range(1, states +1))
        self.random_vars = random_vars
        self.probs = probs
        
    def nodes(self):
        nodes = [(1,1)]
        for i in self.stages[1:]:
            for j in self.states:
                nodes.append((i,j))
        return nodes
    
    def edges(self):
        edges = {}
        for j in self.states:
                edges[(1,1,2,j)] = t_matrix[1][j-1]
        for i in self.stages[2:]:
            for j in self.states:
                for k in self.states:
                    edges[(i-1,j,i,k)] = t_matrix[i-1][j-1][k-1]
        return edges

                
        
    def sample_path(self):
        all_edges = self.edges()
        path = [(1,1)]
        intra = [random.choice(list(random_vars.keys()))  ]
        for i in self.stages[1:]: 
            path.append((i,random.choice(self.states)))
            intra.append(random.choice(list(random_vars.keys())))
        t_prob = []
        for i in self.stages[:-1]:
            t_prob.append(all_edges[path[i-1]+path[i]])
            
        return path, intra, t_prob
    


In [3]:
stages = 3
states = 2

t_matrix = [[1],[0.75,0.25],[[0.75,0.25],[0.25,0.75]]]
intra_probs = {1:1/3, 2:1/3, 3:1/3}
random_vars = {1:{'inflow':0,'fuel':1.5}, 2: {'inflow':50,'fuel':1.0}, 3: {'inflow':100,'fuel':0.75}}

fuel_stage_cost = [50,100,150]

mc = MarkovChain(t_matrix, stages, states, random_vars, intra_probs)



In [4]:
# The subproblem for each node has to be built based on a single intrastage realization 
def subproblem_builder():
    nodes = mc.nodes()
    subproblems = {}
    for i in nodes:

            # placing this outside of the loop an inputing m as a parameter ends up using the same object for all subproblems
            # putting it this way makes sure that each subproblem is a different object, this is a work around just for now.
            m = ConcreteModel()

            #State Variables
            m.volume = Var(within=NonNegativeReals, bounds = (0,200), initialize = 200)
            m.volume_in = Var(within=NonNegativeReals, bounds = (0,200))
            m.cost_to_go = Var(within=NonNegativeReals, initialize=0)
            
            m.volume_x = Param(initialize=0, mutable=True)

            #Local Variables
            m.thermal_generation = Var(within=NonNegativeReals)
            m.hydro_generation = Var(within=NonNegativeReals)
            m.hydro_spill = Var(within=NonNegativeReals)

            #Intrastage Random Variables
            m.inflow = Param(initialize=0, mutable=True)

            #Interstage Random Variables 
            m.fuelcost = Param(initialize=0, mutable=True)
            m.fuelmulti = Param(initialize=0, mutable=True)
            
            m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)

            m.obj = Objective(expr = m.fuelcost*m.fuelmulti * m.thermal_generation + m.cost_to_go, sense=minimize)

            m.con1 = Constraint(expr = m.volume == m.volume_in - m.hydro_generation - m.hydro_spill + m.inflow)
            m.con2 = Constraint(expr = m.hydro_generation + m.thermal_generation == 150)
            m.con3 = Constraint(expr = m.volume_in == m.volume_x)
            m.cuts = ConstraintList()

            subproblems[i] = m
    return subproblems

sb = subproblem_builder()
opt = SolverFactory('gurobi')


import sys, os

# Function that disables printing
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Function that enables printing
def enablePrint():
    sys.stdout = sys.__stdout__


In [5]:
Pij = {((1,1),2,1):0.75,
       ((1,1),2,2):0.25,
       ((2,1),3,1):0.75,
       ((2,1),3,2):0.25,
       ((2,2),3,1):0.25,
       ((2,2),3,2):0.75 } 

import sympy as sp
volume = sp.symbols("volume")



In [6]:
# iteration 1 
# forward pass 

for _ in range(10):
    
    iteration_1_path = mc.sample_path()
    print(iteration_1_path) 
    print(random_vars)
    
    fpi = {}
    
    print("forward pass")
    node, intra = iteration_1_path[0][0], iteration_1_path[1][0]
    
    sb[node].fuelmulti = random_vars[intra]['fuel']
    sb[node].inflow    = random_vars[intra]['inflow']
    sb[node].fuelcost  = fuel_stage_cost[0]
    
    
    sb[node].volume_x = 200
    results = opt.solve(sb[node])
    volume_out = sb[node].volume.value
    
    print("\niteration 1","node", node, "intra", intra) 
    print("volume_in", sb[node].volume_in.value)
    print("volume_out", sb[node].volume.value)
    
    fpi[1] = sb[node].volume.value
    
    node, intra = iteration_1_path[0][1], iteration_1_path[1][1]
    
    sb[node].fuelmulti = random_vars[intra]['fuel']
    sb[node].inflow    = random_vars[intra]['inflow']
    sb[node].fuelcost  = fuel_stage_cost[1]
    
    sb[node].volume_x = volume_out
    results = opt.solve(sb[node])
    volume_out = sb[node].volume.value
    
    print("\niteration 1","node", node, "intra", intra) 
    print("volume_in", sb[node].volume_in.value)
    print("volume_out", sb[node].volume.value)
    
    fpi[2] = sb[node].volume.value
    
    node, intra = iteration_1_path[0][2], iteration_1_path[1][2]
    
    sb[node].fuelmulti = random_vars[intra]['fuel']
    sb[node].inflow    = random_vars[intra]['inflow']
    sb[node].fuelcost  = fuel_stage_cost[2]
    
    sb[node].volume_x = volume_out
    results = opt.solve(sb[node])
    volume_out = sb[node].volume.value
    
    print("\niteration 1","node", node, "intra", intra) 
    print("volume_in", sb[node].volume_in.value)
    print("volume_out", sb[node].volume.value)
    
    fpi[3] = sb[node].volume.value
    
    
    print(iteration_1_path) 
    
    
    # ----------------------------------------------------------------------------  
    
    β = {}
    α = {}
    
    print("\nbackward pass")
    
    opt = SolverFactory('gurobi')
    
    V = {}
    dxdV = {}
    
    node = iteration_1_path[0][1]
    
    # ----------- (3,2) ---------------#
    
    for i in random_vars:
        sb[(3,2)].cost_to_go.fix(0)
        sb[(3,2)].fuelmulti = random_vars[i]['fuel']
        sb[(3,2)].inflow    = random_vars[i]['inflow']
        sb[(3,2)].fuelcost  = fuel_stage_cost[2]
        sb[(3,2)].volume_x = fpi[2]
        results = opt.solve(sb[3,2])
        V[i] = value(sb[3,2].obj)
        dxdV[i] = sb[3,2].dual[sb[3,2].con3]
    intra_probs = {1: 1/2, 2: 1/3, 3: 1/6}
    
    β[2,1,3,2] = Pij[(2,1),3,2]*sum(intra_probs[i]*dxdV[i] for i in random_vars)
    α[2,1,3,2] = Pij[(2,1),3,2]*sum(intra_probs[i]*V[i] for i in random_vars) - β[2,1,3,2]*fpi[2]
    β[2,2,3,2] = Pij[(2,2),3,2]*sum(intra_probs[i]*dxdV[i] for i in random_vars)
    α[2,2,3,2] = Pij[(2,2),3,2]*sum(intra_probs[i]*V[i] for i in random_vars) - β[2,2,3,2]*fpi[2]
    
    # sb[node].cuts.add(expr = sb[node].cost_to_go >= α + β*sb[node].volume)
    
    
    # ----------- (3,1) ---------------#
    
    for i in random_vars:
        sb[(3,1)].cost_to_go.fix(0)
        sb[(3,1)].fuelmulti = random_vars[i]['fuel']
        sb[(3,1)].inflow    = random_vars[i]['inflow']
        sb[(3,1)].fuelcost  = fuel_stage_cost[2]
        sb[(3,1)].volume_x = fpi[2]
        results = opt.solve(sb[3,1])
        V[i] = value(sb[3,1].obj)
        dxdV[i] = sb[3,1].dual[sb[3,1].con3]
    intra_probs = {1: 1/6, 2: 1/3, 3: 1/2}
    
    β[2,1,3,1] = Pij[(2,1),3,1]*sum(intra_probs[i]*dxdV[i] for i in random_vars) 
    α[2,1,3,1] = Pij[(2,1),3,1]*sum(intra_probs[i]*V[i] for i in random_vars) - β[2,1,3,1]*fpi[2]  
    β[2,2,3,1] = Pij[(2,2),3,1]*sum(intra_probs[i]*dxdV[i] for i in random_vars) 
    α[2,2,3,1] = Pij[(2,2),3,1]*sum(intra_probs[i]*V[i] for i in random_vars) - β[2,2,3,1]*fpi[2] 
    
    
    
    sb[2,1].cuts.add(expr = sb[2,1].cost_to_go >= α[2,1,3,1]+α[2,1,3,2] + (β[2,1,3,1]+β[2,1,3,2])*sb[2,1].volume)
    sb[2,2].cuts.add(expr = sb[2,2].cost_to_go >= α[2,2,3,1]+α[2,2,3,2] + (β[2,2,3,1]+β[2,2,3,2])*sb[2,2].volume)
    
    
    # ----------- (2,2) ---------------#
    
    for i in random_vars:
        sb[(2,2)].fuelmulti = random_vars[i]['fuel']
        sb[(2,2)].inflow    = random_vars[i]['inflow']
        sb[(2,2)].fuelcost  = fuel_stage_cost[1]
        sb[(2,2)].volume_x = fpi[1]
        results = opt.solve(sb[2,2])
        V[i] = value(sb[2,2].obj)
        dxdV[i] = sb[2,2].dual[sb[2,2].con3]
    intra_probs = {1: 1/2, 2: 1/3, 3: 1/6}
    
    β[1,1,2,2] = Pij[(1,1),2,2]*sum(intra_probs[i]*dxdV[i] for i in random_vars)
    α[1,1,2,2] = Pij[(1,1),2,2]*sum(intra_probs[i]*V[i] for i in random_vars) - β[1,1,2,2]*fpi[1]
    
    # sb[1,1].cuts.add(expr = sb[1,1].cost_to_go >= α + β*sb[1,1].volume)
    
    # ----------- (2,1) ---------------#
    
    for i in random_vars:
        sb[(2,1)].fuelmulti = random_vars[i]['fuel']
        sb[(2,1)].inflow    = random_vars[i]['inflow']
        sb[(2,1)].fuelcost  = fuel_stage_cost[1]
        sb[(2,1)].volume_x = fpi[1]
        results = opt.solve(sb[2,1])
        V[i] = value(sb[2,1].obj)
        dxdV[i] = sb[2,1].dual[sb[2,1].con3]
    intra_probs = {1: 1/6, 2: 1/3, 3: 1/2}
    
    print(V)
    print(dxdV)
    
    
    β[1,1,2,1] = Pij[(1,1),2,1]*sum(intra_probs[i]*dxdV[i] for i in random_vars) 
    α[1,1,2,1] = Pij[(1,1),2,1]*sum(intra_probs[i]*V[i] for i in random_vars) - β[1,1,2,1]*fpi[1] 
    
    sb[1,1].cuts.add(expr = sb[1,1].cost_to_go >= α[1,1,2,1]+α[1,1,2,2] + (β[1,1,2,1]+β[1,1,2,2])*sb[1,1].volume)
            
    #-------------- (1,1) ---------------$
    
    intra_probs = {1: 1/6, 2: 1/3, 3: 1/2}
    
    
    for i in random_vars:
        sb[1,1].inflow = random_vars[i]['inflow']
        sb[1,1].fuelmulti = random_vars[i]['fuel']
        sb[1,1].fuelcost  = fuel_stage_cost[0]
        results = opt.solve(sb[1,1])
        V[i] = value(sb[1,1].obj)
    
    lower_bound = sum(intra_probs[p]*V[p] for p in random_vars)
    print(lower_bound)

([(1, 1), (2, 1), (3, 1)], [3, 3, 2], [0.75, 0.75])
{1: {'inflow': 0, 'fuel': 1.5}, 2: {'inflow': 50, 'fuel': 1.0}, 3: {'inflow': 100, 'fuel': 0.75}}
forward pass

iteration 1 node (1, 1) intra 3
volume_in 200.0
volume_out 0.0

iteration 1 node (2, 1) intra 3
volume_in 0.0
volume_out 0.0

iteration 1 node (3, 1) intra 2
volume_in 0.0
volume_out 0.0
([(1, 1), (2, 1), (3, 1)], [3, 3, 2], [0.75, 0.75])

backward pass
{1: 38281.25, 2: 23125.0, 3: 11718.75}
{1: -153.125, 2: -153.125, 3: -153.125}
1991.8866886688666


In [ ]:
OK so now 

Find the upper bound and then .... 

write down the whole process and fix your nodes, and then 

try a different markov chain problem see how it goes. 

And also you have to write how did you actually come up to this way of doing it and double check on the 
pseudo codes of the algorithm see if they match what you did just now. 

In [ ]:
sb[1,1].pprint()

In [ ]:
forward_pass_info = {}

def forward_pass(sb, initial_states,forward_pass_info, iteration, sample_path, **kwargs):
        opt = SolverFactory('gurobi')

        # print_log = kwargs.get("detailed_log", False)
        # # replications = kwargs.get("replications", None)

        # if print_log == False:
        #     blockPrint()
            
        print("| Forward Pass")
    
        # print("Sampling Paths\n")
        # print("sampled noise", intra)
        # print("Setting Model Parameters")
        iter_path  = sample_path[0]
        iter_noise = sample_path[1]
    
        for i in iter_path:        
            sb[i].fuelmulti = random_vars[iter_noise[i[0]-1]]['fuel']
            sb[i].inflow   = random_vars[iter_noise[i[0]-1]]['inflow']
            sb[i].fuelcost = fuel_stage_cost[i[0]-1]
            # print("subproblem {} : fuel multiplier = {}, inflow = {}, fuel cost = {}".format(i, sb[i].fuelmulti(), sb[i].inflow(), sb[i].fuelcost()))
       
        node = iter_path[0]

        print("| | Visiting node ", node)
    
        # print("Solving first stage problem with initial state variables with initials {}".format(initial_states))
        sb[node].volume_x = initial_states
        results = opt.solve(sb[node])
        volume_out = sb[node].volume.value
        # print("volume out {}".format(volume_out))
        # print("objective value", value(sb[node].obj))

        forward_pass_info[iteration,node] = {
                                   'inflow': sb[node].inflow.value,
                                   'fuel_cost': sb[node].fuelcost.value,
                                   'fuel_multi': sb[node].fuelmulti.value,
                                   'volume_in': sb[node].volume_in.value, 
                                   'volume_out': sb[node].volume.value,
                                   'cost_to_go': sb[node].cost_to_go.value,
                                   'stage_cost': value(sb[node].obj) - sb[node].cost_to_go.value,
                                   'obj_value' : value(sb[node].obj)}

        print("| | | w = ", sb[node].inflow.value)
        print("| | | x = ", sb[node].volume_in.value)
        print("| | | x' = ", sb[node].volume.value)
        print("| | | C(x,u,w)' = ", value(sb[node].obj) - sb[node].cost_to_go.value)


        node = iter_path[1]

        print("| | Visiting node ", node)

        # print("setting volume in to {} for second subproblem to volume out of first and solving".format(volume_out))
        sb[node].volume_x = volume_out
        results = opt.solve(sb[node])
        volume_out = sb[node].volume.value
        # print("volume out {}".format(volume_out))
        # print("objective value", value(sb[node].obj))

        forward_pass_info[iteration,node] = {
                                   'inflow': sb[node].inflow.value,
                                   'fuel_cost': sb[node].fuelcost.value,
                                   'fuel_multi': sb[node].fuelmulti.value,
                                   'volume_in': sb[node].volume_in.value, 
                                   'volume_out': sb[node].volume.value,
                                   'cost_to_go': sb[node].cost_to_go.value,
                                   'stage_cost': value(sb[node].obj) - sb[node].cost_to_go.value,
                                   'obj_value' : value(sb[node].obj)}

        print("| | | w = ", sb[node].inflow.value)
        print("| | | x = ", sb[node].volume_in.value)
        print("| | | x' = ", sb[node].volume.value)
        print("| | | C(x,u,w)' = ", value(sb[node].obj) - sb[node].cost_to_go.value)


        node = iter_path[2]

        print("| | Visiting node ", node)

        # print("setting volume in for third subproblem to volume out of first and solving")
        sb[node].volume_x = volume_out
        results = opt.solve(sb[node])
        volume_out = sb[node].volume.value
        # print("volume out {}".format(volume_out))
        # print("objective value", value(sb[3].obj))

        forward_pass_info[iteration,node] = {
                                   'inflow': sb[node].inflow.value,
                                   'fuel_cost': sb[node].fuelcost.value,
                                   'fuel_multi': sb[node].fuelmulti.value,
                                   'volume_in': sb[node].volume_in.value, 
                                   'volume_out': sb[node].volume.value,
                                   'cost_to_go': sb[node].cost_to_go.value,
                                   'stage_cost': value(sb[node].obj) - sb[node].cost_to_go.value,
                                   'obj_value' : value(sb[node].obj)}

        print("| | | w = ", sb[node].inflow.value)
        print("| | | x = ", sb[node].volume_in.value)
        print("| | | x' = ", sb[node].volume.value)
        print("| | | C(x,u,w)' = ", value(sb[node].obj) - sb[node].cost_to_go.value)

        # enablePrint()

        # if replications != None:
        #     replication_results = {}
        #     replication_results = forward_simulations(replications, iteration, forward_pass_info, **kwargs)
            
            
        return forward_pass_info
     
#       print("solving subproblem {}, sample noise {}".format(i[1],intra[i[0]]))



In [ ]:
backward_pass_info = {}

def backward_pass(backward_pass_info, iteration, mc, sample_path, **kwargs): 
    print_log = kwargs.get("detailed_log", False)


    iter_path = sample_path[0]
    
    # if print_log == False:
    #     blockPrint()
    
    print("| Backward Pass")

    opt = SolverFactory('gurobi')
    V = {}
    dxdV = {}

    node = iter_path[2]

    print("| | Visiting node ", node)
    print("| | | skipping node because the cost-to-go is 0")

    #Solve the last stage problem for all scenario samples
    sb[node].cost_to_go.fix(0)
    # print("solving for every noise element at node 2")
    for i in random_vars:
        sb[node].inflow = random_vars[i]['inflow']
        sb[node].fuelmulti = random_vars[i]['fuel']
        results = opt.solve(sb[node])
        # print("objective value", value(sb[3].obj))
        # print("dVdx' = ", sb[3].dual[sb[3].con3])
        
        V[i] = value(sb[node].obj)
        dxdV[i] = sb[node].dual[sb[node].con3]
    # Add cut to second stage problem 
    backward_pass_info[iteration,node] = {
                               'inflow': sb[node].inflow.value,
                               'fuel_cost': sb[node].fuelcost.value,
                               'fuel_multi': sb[node].fuelmulti.value,
                               'volume_in': sb[node].volume_in.value, 
                               'volume_out': sb[node].volume.value,
                               'cost_to_go': sb[node].cost_to_go.value,
                               'obj_value' : value(sb[node].obj),
                                'V':V,
                                'dxdV':dxdV}
    

    node = iter_path[1]
    Pij  = [0.75,0.25]
    # print("adding cuts to nodes 2")
    
    sb[node].cuts.add(expr = sb[node].cost_to_go >= Pij[0]*sum(intra_probs[p]*(V[p] + (sb[node].volume-forward_pass_info[iteration,node]['volume_out'])*dxdV[p]) for p in random_vars))
    sb[node].cuts.add(expr = sb[node].cost_to_go >= Pij[1]*sum(intra_probs[p]*(V[p] + (sb[node].volume-forward_pass_info[iteration,node]['volume_out'])*dxdV[p]) for p in random_vars))

    # print(sum(intra_probs[p]*(V[p]) for p in random_vars), sum(intra_probs[p]*dxdV[p] for p in random_vars))
    # print(V, dxdV)


    print("| | Visiting node ", node)
    for i in random_vars:
        print("| | | Solving ϕ = ", random_vars[i]['inflow'])
        print("| | | | V = ", V[i])
        print("| | | | dVdx' =", dxdV[i])
        
    print("| | Adding cut :  ")
    
    # print("solving for every noise element at node 2")
    for i in random_vars:
        sb[node].inflow = random_vars[i]['inflow']
        sb[node].fuelmulti = random_vars[i]['fuel']
        results = opt.solve(sb[node])
        # print("objective value", value(sb[2].obj))
        # print("dVdx' = ", sb[2].dual[sb[2].con3])
        
        V[i] = value(sb[node].obj)
        dxdV[i] = sb[node].dual[sb[node].con3]

    backward_pass_info[iteration,node] = {
                               'inflow': sb[node].inflow.value,
                               'fuel_cost': sb[node].fuelcost.value,
                               'fuel_multi': sb[node].fuelmulti.value,
                               'volume_in': sb[node].volume_in.value, 
                               'volume_out': sb[node].volume.value,
                               'cost_to_go': sb[node].cost_to_go.value,
                               'obj_value' : value(sb[node].obj),
                                'V':V,
                                'dxdV':dxdV}
    

    node = iter_path[0]
    Pij  = [0.75,0.25]

    # print("adding cuts to node 1")
    sb[node].cuts.add(expr = sb[node].cost_to_go >= Pij[0]*sum(intra_probs[p]*(V[p] + (sb[node].volume-forward_pass_info[iteration,node]['volume_out'])*dxdV[p]) for p in random_vars))
    sb[node].cuts.add(expr = sb[node].cost_to_go >= Pij[0]*sum(intra_probs[p]*(V[p] + (sb[node].volume-forward_pass_info[iteration,node]['volume_out'])*dxdV[p]) for p in random_vars))

    # print(sum(intra_probs[p]*(V[p]) for p in random_vars), sum(intra_probs[p]*dxdV[p] for p in random_vars))
    # print(V,dxdV)
#     sb[1].cuts.add(expr = 133.33333333333331*sb[1].volume + sb[1].cost_to_go >= 22500)


    print("| | Visiting node ", node)
    for i in random_vars:
        print("| | | Solving ϕ = ", random_vars[i]['inflow'])
        print("| | | | V = ", V[i])
        print("| | | | dVdx' =", dxdV[i])

    print("| | Adding cut :  ")

    # print("solving for every noise element at node 1 to compute the lower bound")
    for i in random_vars:
        sb[node].inflow = random_vars[i]['inflow']
        sb[node].fuelmulti = random_vars[i]['fuel']
        results = opt.solve(sb[node])
        V[i] = value(sb[node].obj)
    lower_bound = sum(intra_probs[p]*V[p] for p in random_vars)
    
    # enablePrint()
    
    print("| Finished iteration ", iteration)
    print("| | lower bound = ", lower_bound)

    backward_pass_info[iteration,node] = {
                           'inflow': sb[node].inflow.value,
                           'fuel_cost': sb[node].fuelcost.value,
                           'fuel_multi': sb[node].fuelmulti.value,
                           'volume_in': sb[node].volume_in.value, 
                           'volume_out': sb[node].volume.value,
                           'cost_to_go': sb[node].cost_to_go.value,
                           'obj_value' : value(sb[node].obj),
                            'V':V,
                            'dxdV':dxdV,
                            'lower_bound': lower_bound}

    
    return backward_pass_info

In [ ]:
# iteration = 1

# sample_path = mc.sample_path()

# forward_pass_info = forward_pass(sb, 200 ,forward_pass_info, iteration, sample_path)
# backward_pass(backward_pass_info, iteration, mc, sample_path)



In [ ]:
V

In [ ]:
# iteration 1 
# forward pass 

for _ in range(20):

    iteration_1_path = mc.sample_path()
    print(iteration_1_path) 
    print(random_vars)
    
    fpi = {}
    
    print("forward pass")
    node, intra = iteration_1_path[0][0], iteration_1_path[1][0]
    
    sb[node].fuelmulti = random_vars[intra]['fuel']
    sb[node].inflow    = random_vars[intra]['inflow']
    sb[node].fuelcost  = fuel_stage_cost[0]
    
    
    sb[node].volume_x = 200
    results = opt.solve(sb[node])
    volume_out = sb[node].volume.value
    
    print("\niteration 1","node", node, "intra", intra) 
    print("volume_in", sb[node].volume_in.value)
    print("volume_out", sb[node].volume.value)
    
    fpi[1] = sb[node].volume.value
    
    node, intra = iteration_1_path[0][1], iteration_1_path[1][1]
    
    sb[node].fuelmulti = random_vars[intra]['fuel']
    sb[node].inflow    = random_vars[intra]['inflow']
    sb[node].fuelcost  = fuel_stage_cost[1]
    
    sb[node].volume_x = volume_out
    results = opt.solve(sb[node])
    volume_out = sb[node].volume.value
    
    print("\niteration 1","node", node, "intra", intra) 
    print("volume_in", sb[node].volume_in.value)
    print("volume_out", sb[node].volume.value)
    
    fpi[2] = sb[node].volume.value
    
    node, intra = iteration_1_path[0][2], iteration_1_path[1][2]
    
    sb[node].fuelmulti = random_vars[intra]['fuel']
    sb[node].inflow    = random_vars[intra]['inflow']
    sb[node].fuelcost  = fuel_stage_cost[2]
    
    sb[node].volume_x = volume_out
    results = opt.solve(sb[node])
    volume_out = sb[node].volume.value
    
    print("\niteration 1","node", node, "intra", intra) 
    print("volume_in", sb[node].volume_in.value)
    print("volume_out", sb[node].volume.value)
    
    fpi[3] = sb[node].volume.value
    
    
    print(iteration_1_path) 


# ----------------------------------------------------------------------------  

    print("\nbackward pass")
    
    opt = SolverFactory('gurobi')
    
    V = {}
    dxdV = {}
    
    node = iteration_1_path[0][1]
    
    # ----------- (3,2) ---------------#
    
    for i in random_vars:
        sb[(3,2)].cost_to_go.fix(0)
        sb[(3,2)].fuelmulti = random_vars[i]['fuel']
        sb[(3,2)].inflow    = random_vars[i]['inflow']
        sb[(3,2)].fuelcost  = fuel_stage_cost[2]
        sb[(3,2)].volume_x = fpi[2]
        results = opt.solve(sb[3,2])
        V[i] = value(sb[3,2].obj)
        dxdV[i] = sb[3,2].dual[sb[3,2].con3]
    
    intra_probs = {1: 1/2, 2: 1/3, 3: 1/6}
    sp_cut = sp.simplify(Pij[(2,1),3,2]*sum(intra_probs[p]*(V[p] + (volume - fpi[2])*dxdV[p]) for p in random_vars)).as_coefficients_dict()
    sb[(2,1)].cuts.add(expr = sb[(2,1)].cost_to_go >= float(sp_cut[1]) + float(sp_cut[volume])*sb[(2,1)].volume)
    sp_cut = sp.simplify(Pij[(2,2),3,2]*sum(intra_probs[p]*(V[p] + (volume - fpi[2])*dxdV[p]) for p in random_vars)).as_coefficients_dict()
    sb[(2,2)].cuts.add(expr = sb[(2,2)].cost_to_go >= float(sp_cut[1]) + float(sp_cut[volume])*sb[(2,2)].volume)


    print(V)

    
    # sb[node].cuts.add(expr = sb[node].cost_to_go >= Pij[node,3,2]*sum(intra_probs[p]*(V[p] + (sb[node].volume - fpi[2])*dxdV[p]) for p in random_vars))
    
    
    
    # ----------- (3,1) ---------------#
    
    for i in random_vars:
        sb[(3,1)].cost_to_go.fix(0)
        sb[(3,1)].fuelmulti = random_vars[i]['fuel']
        sb[(3,1)].inflow    = random_vars[i]['inflow']
        sb[(3,1)].fuelcost  = fuel_stage_cost[2]
        sb[(3,1)].volume_x = fpi[2]
        results = opt.solve(sb[3,1])
        V[i] = value(sb[3,1].obj)
        dxdV[i] = sb[3,1].dual[sb[3,1].con3]
    intra_probs = {1: 1/6, 2: 1/3, 3: 1/2}
    sp_cut = sp.simplify(Pij[(2,1),3,1]*sum(intra_probs[p]*(V[p] + (volume - fpi[2])*dxdV[p]) for p in random_vars)).as_coefficients_dict()
    sb[(2,1)].cuts.add(expr = sb[(2,1)].cost_to_go >= float(sp_cut[1]) + float(sp_cut[volume])*sb[(2,1)].volume)
    sp_cut = sp.simplify(Pij[(2,2),3,1]*sum(intra_probs[p]*(V[p] + (volume - fpi[2])*dxdV[p]) for p in random_vars)).as_coefficients_dict()
    sb[(2,2)].cuts.add(expr = sb[(2,2)].cost_to_go >= float(sp_cut[1]) + float(sp_cut[volume])*sb[(2,2)].volume)


    print(V)

    
    # sb[node].cuts.add(expr = sb[node].cost_to_go >= Pij[node,3,1]*sum(intra_probs[p]*(V[p] + (sb[node].volume - fpi[2])*dxdV[p]) for p in random_vars))
    
    
    
    # ----------- (2,2) ---------------#
    
    for i in random_vars:
        sb[(2,2)].fuelmulti = random_vars[i]['fuel']
        sb[(2,2)].inflow    = random_vars[i]['inflow']
        sb[(2,2)].fuelcost  = fuel_stage_cost[1]
        sb[(2,2)].volume_x = fpi[1]
        results = opt.solve(sb[2,2])
        V[i] = value(sb[2,2].obj)
        dxdV[i] = sb[2,2].dual[sb[2,2].con3]
    intra_probs = {1: 1/2, 2: 1/3, 3: 1/6}
    sp_cut = sp.simplify(Pij[(1,1),2,2]*sum(intra_probs[p]*(V[p] + (volume - fpi[1])*dxdV[p]) for p in random_vars)).as_coefficients_dict()
    sb[(1,1)].cuts.add(expr = sb[(1,1)].cost_to_go >= float(sp_cut[1]) + float(sp_cut[volume])*sb[(1,1)].volume)

    print(V)
    
    
    # sb[(1,1)].cuts.add(expr = sb[(1,1)].cost_to_go >= Pij[(1,1),2,2]*sum(intra_probs[p]*(V[p] + (sb[(1,1)].volume - fpi[1])*dxdV[p]) for p in random_vars))
    
    # ----------- (2,1) ---------------#
    
    for i in random_vars:
        sb[(2,1)].fuelmulti = random_vars[i]['fuel']
        sb[(2,1)].inflow    = random_vars[i]['inflow']
        sb[(2,1)].fuelcost  = fuel_stage_cost[1]
        sb[(2,1)].volume_x = fpi[1]
        results = opt.solve(sb[2,1])
        V[i] = value(sb[2,1].obj)
        dxdV[i] = sb[2,1].dual[sb[2,1].con3]
    intra_probs = {1: 1/6, 2: 1/3, 3: 1/2}
    sp_cut = sp.simplify(Pij[(1,1),2,1]*sum(intra_probs[p]*(V[p] + (volume - fpi[1])*dxdV[p]) for p in random_vars)).as_coefficients_dict()
    sb[(1,1)].cuts.add(expr = sb[(1,1)].cost_to_go >= float(sp_cut[1]) + float(sp_cut[volume])*sb[(1,1)].volume)
    
    print(V)

        
    # sb[(1,1)].cuts.add(expr = sb[(1,1)].cost_to_go >= Pij[(1,1),2,1]*sum(intra_probs[p]*(V[p] + (sb[(1,1)].volume - fpi[1])*dxdV[p]) for p in random_vars))
    
    #-------------- (1,1) ---------------$
    
    intra_probs = {1: 1/2, 2: 1/3, 3: 1/6}
    
    
    for i in random_vars:
        sb[1,1].inflow = random_vars[i]['inflow']
        sb[1,1].fuelmulti = random_vars[i]['fuel']
        sb[1,1].fuelcost  = fuel_stage_cost[0]
        results = opt.solve(sb[1,1])
        V[i] = value(sb[1,1].obj)
    
    lower_bound = sum(intra_probs[p]*V[p] for p in random_vars)
    print(lower_bound)

In [ ]:
iteration_1_path = mc.sample_path()
print(iteration_1_path) 
print(random_vars)


# iteration 1 
# forward pass 

fpi = {}

print("forward pass")
node, intra = iteration_1_path[0][0], iteration_1_path[1][0]

sb[node].fuelmulti = random_vars[intra]['fuel']
sb[node].inflow    = random_vars[intra]['inflow']
sb[node].fuelcost  = fuel_stage_cost[0]


sb[node].volume_x = 200
results = opt.solve(sb[node])
volume_out = sb[node].volume.value

print("\niteration 1","node", node, "intra", intra) 
print("volume_in", sb[node].volume_in.value)
print("volume_out", sb[node].volume.value)

fpi[1] = sb[node].volume.value

node, intra = iteration_1_path[0][1], iteration_1_path[1][1]

sb[node].fuelmulti = random_vars[intra]['fuel']
sb[node].inflow    = random_vars[intra]['inflow']
sb[node].fuelcost  = fuel_stage_cost[1]

sb[node].volume_x = volume_out
results = opt.solve(sb[node])
volume_out = sb[node].volume.value

print("\niteration 1","node", node, "intra", intra) 
print("volume_in", sb[node].volume_in.value)
print("volume_out", sb[node].volume.value)

fpi[2] = sb[node].volume.value

print("\nbackward pass")

opt = SolverFactory('gurobi')

V = {}
dxdV = {}
for i in random_vars:
    sb[(3,2)].cost_to_go.fix(0)
    sb[(3,2)].fuelmulti = random_vars[i]['fuel']
    sb[(3,2)].inflow    = random_vars[i]['inflow']
    sb[(3,2)].fuelcost  = fuel_stage_cost[2]
    results = opt.solve(sb[3,2])
    V[i] = value(sb[3,2].obj)
    dxdV[i] = sb[3,2].dual[sb[3,2].con3]

node = iteration_1_path[0][1]

intra_probs = {1: 1/2, 2: 1/3, 3: 1/6}

    
sb[node].cuts.add(expr = sb[node].cost_to_go >= Pij[node,3,2]*sum(intra_probs[p]*(V[p] + (sb[node].volume-fpi[2])*dxdV[p]) for p in random_vars))


for i in random_vars:
    sb[(3,1)].cost_to_go.fix(0)
    sb[(3,1)].fuelmulti = random_vars[i]['fuel']
    sb[(3,1)].inflow    = random_vars[i]['inflow']
    sb[(3,1)].fuelcost  = fuel_stage_cost[2]
    results = opt.solve(sb[3,1])
    V[i] = value(sb[3,1].obj)
    dxdV[i] = sb[3,1].dual[sb[3,1].con3]

intra_probs = {1: 1/6, 2: 1/3, 3: 1/2}


    
sb[node].cuts.add(expr = sb[node].cost_to_go >= Pij[node,3,1]*sum(intra_probs[p]*(V[p] + (sb[node].volume-fpi[2])*dxdV[p]) for p in random_vars))


for i in random_vars:
    sb[(2,2)].fuelmulti = random_vars[i]['fuel']
    sb[(2,2)].inflow    = random_vars[i]['inflow']
    sb[(2,2)].fuelcost  = fuel_stage_cost[1]
    results = opt.solve(sb[2,2])
    V[i] = value(sb[2,2].obj)
    dxdV[i] = sb[2,2].dual[sb[2,2].con3]


intra_probs = {1: 1/2, 2: 1/3, 3: 1/6}

    
sb[(1,1)].cuts.add(expr = sb[(1,1)].cost_to_go >= Pij[(1,1),2,2]*sum(intra_probs[p]*(V[p] + (sb[(1,1)].volume-fpi[1])*dxdV[p]) for p in random_vars))


for i in random_vars:
    sb[(2,1)].fuelmulti = random_vars[i]['fuel']
    sb[(2,1)].inflow    = random_vars[i]['inflow']
    sb[(2,1)].fuelcost  = fuel_stage_cost[1]
    results = opt.solve(sb[2,1])
    V[i] = value(sb[2,1].obj)
    dxdV[i] = sb[2,1].dual[sb[2,1].con3]

intra_probs = {1: 1/6, 2: 1/3, 3: 1/2}


    
sb[(1,1)].cuts.add(expr = sb[(1,1)].cost_to_go >= Pij[(1,1),2,1]*sum(intra_probs[p]*(V[p] + (sb[(1,1)].volume-fpi[1])*dxdV[p]) for p in random_vars))


for i in random_vars:
    sb[1,1].inflow = random_vars[i]['inflow']
    sb[1,1].fuelmulti = random_vars[i]['fuel']
    sb[1,1].fuelcost  = fuel_stage_cost[0]
    results = opt.solve(sb[1,1])
    V[i] = value(sb[1,1].obj)

lower_bound = sum(intra_probs[p]*V[p] for p in random_vars)
print(lower_bound)

In [ ]:
# iteration 1 
# forward pass 

for _ in range(30):

    iteration_1_path = mc.sample_path()
    print(iteration_1_path) 
    print(random_vars)
    
    fpi = {}
    
    print("forward pass")
    node, intra = iteration_1_path[0][0], iteration_1_path[1][0]
    
    sb[node].fuelmulti = random_vars[intra]['fuel']
    sb[node].inflow    = random_vars[intra]['inflow']
    sb[node].fuelcost  = fuel_stage_cost[0]
    
    
    sb[node].volume_x = 200
    results = opt.solve(sb[node])
    volume_out = sb[node].volume.value
    
    print("\niteration 1","node", node, "intra", intra) 
    print("volume_in", sb[node].volume_in.value)
    print("volume_out", sb[node].volume.value)
    
    fpi[1] = sb[node].volume.value
    
    node, intra = iteration_1_path[0][1], iteration_1_path[1][1]
    
    sb[node].fuelmulti = random_vars[intra]['fuel']
    sb[node].inflow    = random_vars[intra]['inflow']
    sb[node].fuelcost  = fuel_stage_cost[1]
    
    sb[node].volume_x = volume_out
    results = opt.solve(sb[node])
    volume_out = sb[node].volume.value
    
    print("\niteration 1","node", node, "intra", intra) 
    print("volume_in", sb[node].volume_in.value)
    print("volume_out", sb[node].volume.value)
    
    fpi[2] = sb[node].volume.value
    
    node, intra = iteration_1_path[0][2], iteration_1_path[1][2]
    
    sb[node].fuelmulti = random_vars[intra]['fuel']
    sb[node].inflow    = random_vars[intra]['inflow']
    sb[node].fuelcost  = fuel_stage_cost[2]
    
    sb[node].volume_x = volume_out
    results = opt.solve(sb[node])
    volume_out = sb[node].volume.value
    
    print("\niteration 1","node", node, "intra", intra) 
    print("volume_in", sb[node].volume_in.value)
    print("volume_out", sb[node].volume.value)
    
    fpi[3] = sb[node].volume.value
    
    
    print(iteration_1_path) 


# ----------------------------------------------------------------------------  

    print("\nbackward pass")
    
    opt = SolverFactory('gurobi')
    
    V = {}
    dxdV = {}
    
    node = iteration_1_path[0][1]
    
    # ----------- (3,2) ---------------#
    
    for i in random_vars:
        sb[(3,2)].cost_to_go.fix(0)
        sb[(3,2)].fuelmulti = random_vars[i]['fuel']
        sb[(3,2)].inflow    = random_vars[i]['inflow']
        sb[(3,2)].fuelcost  = fuel_stage_cost[2]
        sb[(3,2)].volume_x = fpi[2]
        results = opt.solve(sb[3,2])
        V[i] = value(sb[3,2].obj)
        dxdV[i] = sb[3,2].dual[sb[3,2].con3]
    intra_probs = {1: 1/2, 2: 1/3, 3: 1/6}
    sp_cut = sp.simplify(Pij[node,3,2]*sum(intra_probs[p]*(V[p] + (volume - fpi[2])*dxdV[p]) for p in random_vars)).as_coefficients_dict()
    sb[node].cuts.add(expr = sb[node].cost_to_go >= float(sp_cut[1]) + float(sp_cut[volume])*sb[node].volume)
    
    
    # sb[node].cuts.add(expr = sb[node].cost_to_go >= Pij[node,3,2]*sum(intra_probs[p]*(V[p] + (sb[node].volume - fpi[2])*dxdV[p]) for p in random_vars))
    
    
    
    # ----------- (3,1) ---------------#
    
    for i in random_vars:
        sb[(3,1)].cost_to_go.fix(0)
        sb[(3,1)].fuelmulti = random_vars[i]['fuel']
        sb[(3,1)].inflow    = random_vars[i]['inflow']
        sb[(3,1)].fuelcost  = fuel_stage_cost[2]
        sb[(3,1)].volume_x = fpi[2]
        results = opt.solve(sb[3,1])
        V[i] = value(sb[3,1].obj)
        dxdV[i] = sb[3,1].dual[sb[3,1].con3]
    intra_probs = {1: 1/6, 2: 1/3, 3: 1/2}
    sp_cut = sp.simplify(Pij[node,3,1]*sum(intra_probs[p]*(V[p] + (volume - fpi[2])*dxdV[p]) for p in random_vars)).as_coefficients_dict()
    sb[node].cuts.add(expr = sb[node].cost_to_go >= float(sp_cut[1]) + float(sp_cut[volume])*sb[node].volume)
    
    
    # sb[node].cuts.add(expr = sb[node].cost_to_go >= Pij[node,3,1]*sum(intra_probs[p]*(V[p] + (sb[node].volume - fpi[2])*dxdV[p]) for p in random_vars))
    
    
    
    # ----------- (2,2) ---------------#
    
    for i in random_vars:
        sb[(2,2)].fuelmulti = random_vars[i]['fuel']
        sb[(2,2)].inflow    = random_vars[i]['inflow']
        sb[(2,2)].fuelcost  = fuel_stage_cost[1]
        sb[(2,2)].volume_x = fpi[1]
        results = opt.solve(sb[2,2])
        V[i] = value(sb[2,2].obj)
        dxdV[i] = sb[2,2].dual[sb[2,2].con3]
    intra_probs = {1: 1/2, 2: 1/3, 3: 1/6}
    sp_cut = sp.simplify(Pij[(1,1),2,2]*sum(intra_probs[p]*(V[p] + (volume - fpi[1])*dxdV[p]) for p in random_vars)).as_coefficients_dict()
    sb[(1,1)].cuts.add(expr = sb[(1,1)].cost_to_go >= float(sp_cut[1]) + float(sp_cut[volume])*sb[(1,1)].volume)
    
    
    
    # sb[(1,1)].cuts.add(expr = sb[(1,1)].cost_to_go >= Pij[(1,1),2,2]*sum(intra_probs[p]*(V[p] + (sb[(1,1)].volume - fpi[1])*dxdV[p]) for p in random_vars))
    
    # ----------- (2,1) ---------------#
    
    for i in random_vars:
        sb[(2,1)].fuelmulti = random_vars[i]['fuel']
        sb[(2,1)].inflow    = random_vars[i]['inflow']
        sb[(2,1)].fuelcost  = fuel_stage_cost[1]
        sb[(2,1)].volume_x = fpi[1]
        results = opt.solve(sb[2,1])
        V[i] = value(sb[2,1].obj)
        dxdV[i] = sb[2,1].dual[sb[2,1].con3]
    intra_probs = {1: 1/6, 2: 1/3, 3: 1/2}
    sp_cut = sp.simplify(Pij[(1,1),2,1]*sum(intra_probs[p]*(V[p] + (volume - fpi[1])*dxdV[p]) for p in random_vars)).as_coefficients_dict()
    sb[(1,1)].cuts.add(expr = sb[(1,1)].cost_to_go >= float(sp_cut[1]) + float(sp_cut[volume])*sb[(1,1)].volume)
    
    
        
    # sb[(1,1)].cuts.add(expr = sb[(1,1)].cost_to_go >= Pij[(1,1),2,1]*sum(intra_probs[p]*(V[p] + (sb[(1,1)].volume - fpi[1])*dxdV[p]) for p in random_vars))
    
    #-------------- (1,1) ---------------$
    
    intra_probs = {1: 1/6, 2: 1/3, 3: 1/2}
    
    
    for i in random_vars:
        sb[1,1].inflow = random_vars[i]['inflow']
        sb[1,1].fuelmulti = random_vars[i]['fuel']
        sb[1,1].fuelcost  = fuel_stage_cost[0]
        results = opt.solve(sb[1,1])
        V[i] = value(sb[1,1].obj)
    
    lower_bound = sum(intra_probs[p]*V[p] for p in random_vars)
    print(lower_bound)

In [ ]:
Everytime I am getting an entiarly different result. There is something extremely wrong with the way I am setting up 
this problem and I am making the cuts.

In [ ]:
for i in sb[1,1].cuts:
    print(sb[1,1].cuts[i].expr)

In [ ]:
I think i know what the problem is now, the problem is that I am not setting Volume In Again, to the subproblems, but am i not though? 
Maybe I am 

In [ ]:
sp_cut = sp.simplify(Pij[(2,2),3,1]*sum(intra_probs[p]*(V[p] + (volume - fpi[2])*dxdV[p]) for p in random_vars)).as_coefficients_dict()

In [ ]:
sb[2,2].pprint()

In [ ]:
are the inflow and fuel cost noise probabilities different? should they be? 

no i think i know why, i think its because the cuts are not universally shared. 